# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weatherPyCSV=pd.read_csv('../WeatherPy/Weather_Data.csv')
weatherPyCSV

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,nome,64.50,-165.41,34.00,86,90,4.70,US,1603842620
1,libreville,0.39,9.45,77.00,100,75,4.70,GA,1603842702
2,cromarty,57.68,-4.03,44.01,100,100,3.36,GB,1603842703
3,punta arenas,-53.15,-70.92,46.40,61,0,16.11,CL,1603842703
4,kota kinabalu,5.97,116.07,77.00,78,20,5.82,MY,1603842703
...,...,...,...,...,...,...,...,...,...
557,hokitika,-42.72,170.97,58.64,68,91,7.78,NZ,1603842794
558,mantua,45.17,10.78,46.99,93,0,4.70,IT,1603842794
559,flinders,-34.58,150.86,66.99,78,84,1.01,AU,1603842795
560,santander,43.46,-3.80,59.00,100,75,5.82,ES,1603842795


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = weatherPyCSV[["Lat", "Lng"]].astype(float)

humidity = weatherPyCSV["Humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=weatherPyCSV["Humidity"].max().astype(float),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#drop any null values
cleanedWeather=weatherPyCSV.dropna()
#create data frame with temps between 65 and 80 degrees
idealWeather=cleanedWeather.loc[(cleanedWeather["Max Temp"]>65) & (cleanedWeather["Max Temp"]<80)]
#narrow down above data frame to also have cloudiness less than 5% and wind speed less than 5mph
idealWeatherDF=idealWeather.loc[(idealWeather["Cloudiness"]<5) & (idealWeather["Wind Speed"]<5)]
idealWeatherDF

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
42,souillac,-20.52,57.52,70.00,80,4,1.99,MU,1603842709
49,mahebourg,-20.41,57.70,70.00,80,3,1.99,MU,1603842711
63,saint-philippe,-21.36,55.77,71.60,64,0,3.36,RE,1603842505
64,bara,10.37,10.73,75.72,32,0,4.79,NG,1603842713
150,tura,25.52,90.22,67.62,88,0,4.14,IN,1603842727
183,indi,17.17,75.97,71.37,78,1,3.53,IN,1603842733
213,moreira sales,-24.06,-53.01,67.10,59,0,3.42,BR,1603842737
220,ambilobe,-13.20,49.05,70.88,88,1,3.09,MG,1603842739
291,laguna,38.42,-121.42,75.00,18,1,3.36,US,1603842477
297,abu dhabi,24.47,54.37,77.00,65,0,4.70,AE,1603842648


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
#reset index
hotel_df=idealWeatherDF[["City","Lat","Lng","Country"]].reset_index()
#add column for Hotel Name
hotel_df["Hotel Name"]=""
hotel_df

,index,City,Lat,Lng,Country,Hotel Name
0,42,souillac,-20.52,57.52,MU,
1,49,mahebourg,-20.41,57.70,MU,
2,63,saint-philippe,-21.36,55.77,RE,
3,64,bara,10.37,10.73,NG,
4,150,tura,25.52,90.22,IN,
5,183,indi,17.17,75.97,IN,
6,213,moreira sales,-24.06,-53.01,BR,
7,220,ambilobe,-13.20,49.05,MG,
8,291,laguna,38.42,-121.42,US,
9,297,abu dhabi,24.47,54.37,AE,


In [8]:
#set parameters
params={"radius":5000,"types":"lodging","key":g_key}

for index, row in hotel_df.iterrows():
    #add parameter for lat and lng
    params["location"]=f"{row['Lat']},{row['Lng']}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    #send request
    name_address = requests.get(base_url, params=params).json()
    
    try:
        #store the name of the first hotel that is in the request response
        hotel_df.loc[index, 'Hotel Name'] = name_address['results'][0]['name']
        print(f"The hotel closest to  {hotel_df.loc[index, 'City'] } is {name_address['results'][0]['name']}.")
    except:
        #store N/A since no hotel was found
        hotel_df.loc[index, 'Hotel Name'] = "N/A"
        print(f"There is no hotel within 5000km of {hotel_df.loc[index, 'City']}.")

The hotel closest to  souillac is Bils' Villa.
The hotel closest to  mahebourg is Shandrani Beachcomber Resort & Spa.
The hotel closest to  saint-philippe is Chambres d'hôte "La Trinité".
There is no hotel within 5000km of bara.
The hotel closest to  tura is Hotel RIKMAN Continental.
The hotel closest to  indi is Hotel West End.
The hotel closest to  moreira sales is Portal Moreira Sales.
The hotel closest to  ambilobe is Hôtel & Restaurant DIANA Ambilobe.
The hotel closest to  laguna is Holiday Inn Express & Suites Elk Grove Central - Hwy 99.
The hotel closest to  abu dhabi is Beach Rotana.
The hotel closest to  cap malheureux is Pereybere Hotel & Spa.
The hotel closest to  saint-pierre is Lindsey Hôtel.
The hotel closest to  bambous virieux is Casa Tia Villa.
The hotel closest to  hwange is Hwange Colliery Guest House.
The hotel closest to  grand baie is Trou aux Biches Beachcomber Golf Resort & Spa.
The hotel closest to  raghogarh is H21, JUET.
The hotel closest to  mecca is Hotel F

In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))